In [107]:
import pandas as pd
import numpy as np    
import re   # Searching the text doucument in data
from nltk.corpus import stopwords   # Natural Language Toolkit
from nltk.stem.porter import PorterStemmer   # remove prefix and suffix and return root word
from sklearn.feature_extraction.text import TfidfVectorizer
# Convert Text to numerical vector
# Term Frequency-Inverse Document Frequency)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Data Pre-Processing

In [108]:
news = pd.read_csv("train.csv")

In [109]:
news.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [110]:
news.shape

(20800, 5)

In [111]:
import nltk
nltk.download('stopwords') 

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/5fffed6a-e895-447b-8444-
[nltk_data]     72e299d7432d/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [112]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [113]:
news.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [114]:
# 1 : Fake News
# 0 : Real News 

## Counting the missing values


In [115]:
news.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

### filling  The Misssing Value with empty String

In [116]:
news = news.fillna('')

In [117]:
news.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

## Merging the author name and title Column

In [118]:
news['content'] = news['author'] + ' ' + news['title']

In [119]:
news.head(2)

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."


In [120]:
news.iloc[0].content

'Darrell Lucus House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

# Seprating the data And Label

In [121]:
X = news.drop(columns='label',axis=1)
Y = news['label']

In [122]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

# Stemming 

In [123]:
# "Stemming" is the process of reducing words to their root or base form
# (e.g., "running" becomes "run"). PorterStemmer is a specific algorithm for stemming.

In [124]:
port_stem = PorterStemmer()

In [125]:
from nltk.corpus import stopwords

def stemming(content):
    stemed_cont = re.sub('[^a-zA-z]',' ',content)
    stemed_cont = stemed_cont.lower()
    stemed_cont = stemed_cont.split()
    stemed_cont = [port_stem.stem(word) for word in stemed_cont if not word in stopwords.words('english')]  # Changed .word() to .words()
    stemed_cont = ' '.join(stemed_cont)
    return stemed_cont


In [126]:
news['content'] = news['content'].apply(stemming)

In [127]:
print(news['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


# Seprating the data And Label

In [128]:
news.head(2)

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,darrel lucu hous dem aid even see comey letter...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,daniel j flynn flynn hillari clinton big woman...


In [129]:
X = news['content'].values
Y = news['label'].values

In [130]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [131]:
print(Y)

[1 0 1 ... 0 1 1]


In [132]:
X.shape

(20800,)

In [133]:
Y.shape

(20800,)

# TFidf - Vectorizer

In [134]:
## This is converting textual data into Numerical Data

In [135]:
vect = TfidfVectorizer()
vect.fit(X)
X = vect.transform(X)

In [136]:
print(X)

  (0, 15697)	0.28485063562728646
  (0, 13480)	0.2565896679337957
  (0, 8915)	0.3635963806326075
  (0, 8636)	0.29212514087043684
  (0, 7698)	0.24785219520671603
  (0, 7012)	0.21874169089359144
  (0, 4979)	0.23331696690935097
  (0, 3795)	0.2705332480845492
  (0, 3603)	0.3598939188262559
  (0, 2962)	0.2468450128533713
  (0, 2485)	0.3676519686797209
  (0, 268)	0.2701012497770876
  (1, 16812)	0.30071745655510157
  (1, 6823)	0.19046601982968486
  (1, 5510)	0.7143299355715573
  (1, 3571)	0.26373768806048464
  (1, 2816)	0.19094574062359204
  (1, 2224)	0.3827320386859759
  (1, 1895)	0.15521974226349364
  (1, 1498)	0.2939891562094648
  (2, 15622)	0.41544962664721613
  (2, 9625)	0.49351492943649944
  (2, 5975)	0.3474613386728292
  (2, 5396)	0.3866530551182615
  (2, 3106)	0.46097489583229645
  :	:
  (20797, 13128)	0.2482526352197606
  (20797, 12350)	0.27263457663336677
  (20797, 12144)	0.24778257724396507
  (20797, 10311)	0.08038079000566466
  (20797, 9593)	0.174553480255222
  (20797, 9523)	0.2954

# Now The Model

## Splitting the Training and Testing Data Set 

In [141]:
X_train,X_test,Y_train,Y_test = train_test_split(X ,Y , test_size = 0.2 , stratify=Y , random_state=2)

# Training Logistic Regression Model 

In [142]:
model = LogisticRegression()

In [143]:
model.fit(X_train , Y_train)

LogisticRegression()

# Finding The Accuracy Score : 

## Accuracy Score In The Training Data

In [144]:
X_train_Pred = model.predict(X_train)
X_train_acc = accuracy_score(X_train_Pred , Y_train)

In [145]:
print( X_train_acc)

0.9864783653846154


## Accuracy Score In Test Data

In [146]:
X_test_Pred = model.predict(X_test)
X_test_acc = accuracy_score(X_test_Pred , Y_test)

In [147]:
print( X_test_acc)

0.9790865384615385


# Making A Predictive System

In [153]:
X_new_news = X_test[0]

prediction = model.predict(X_new_news)

print(prediction)


if (prediction[0] == 0):
    print("The News is Real")
else:
    print("The News is Fake")

[1]
The News is Fake


In [155]:
print(Y_test[0])

1
